# Look at the Big Picture

California 인구조사 데이터를 사용해 California의 housing price의 모델을 만들어본다.

이 데이터는 California의 block group에 대한 population, median income, median housing price 등의 지표를 가지고 있다. 여기서 block group은 US Census Bureau가 발표한 샘플 데이터에 대한 최소한의 지형적 단위이다.(한 block group은 보통 600~3000명의 인구를 가진다.) 여기서는 이를 간단히 "districts"라고 부르기로 한다.

모델은 이 데이터로부터 학습해서 어느 district에서나 다른 지표들로부터 median housing price를 예측할 수 있어야 한다.

## Frame the Problem

모델을 만들기 전에 확실히 해야 하는 것은 정확한 목표이다.

모델이 어떻게 사용될 것이고 이득을 얻을 것인지는 매우 중요하다. 이는 어떻게 문제를 정의하고, 어떤 알고리즘을 선택하고, 어떠한 성능을 측정해서 모델을 평가할 것이며, 모델 수정에 얼마나 많은 노력을 기울여야 하는지를 결정하기 때문이다.

모델의 출력(district의 median housing price 예측)이 다른 머신러닝 시스템에 다른 많은 신호들과 함께 주입된다고 하자. 뒤따르는 시스템은 해당 지역에 투자할 가치가 있는지 아닌지를 결정한다. 다음 그림과 같은 방식이다.

<br/>
<img src="./images/machine_learning_pipeline_for_real_estate_investments.png" alt="machine_learning_pipeline_for_real_estate_investments" width="80%">
<br/>

다음으로 고려해야 할 것은 현재의 솔루션이 어떤지 보는것이다.

이는 보통 reference perfornance로 사용되며, 어떻게 문제를 해결해야 할지에 도움이 된다. 만약, 현재의 솔루션이 전문가로부터 직접 district의 median housing price의 최신 정보를 수집하는 방식이라고 한다면 이는 많은 비용과 시간이 소요되는 방식이고 정확하게 예측하기 어려울 것이다. 따라서, 머신러닝 모델을 사용하는 것이 타당하다고 볼 수 있다.

이제 시스템을 설계할 차례이다.

이 문제는 label(district의 median housing price)이 주어지므로 supervised learing이다. 또한, 값을 예측하기 때문에 전형적인 regression 문제이다. 더 구체적으로는 multiple features로부터 예측하기 때문에 multiple regression이라고 할 수 있다. 마지막으로, 연속적인 데이터 흐름이 없고 데이터의 양이 적으므로 batch learning을 사용해도 무관하다.

따라서, 이 문제는 다음과 같이 정리할 수 있다.
- supervised learning
- multiple regression
- batch learning

### 참고) Pipelines
Data pipeline은 data processing components의 sequence를 의미한다. 머신러닝 시스템에서는 데이터를 조작하고 변환하는 일이 많으므로 pipeline은 아주 흔하게 사용된다.

Component들은 일반적으로 비동기적이다. 각 component는 많은 데이터를 가져가서 처리하고 그 결과를 다른 저장소로 보낸다. 그리고 일정시간 후 pipeline의 다음 컴포넌트는 그 데이터를 가져가서 처리하고 그 결과를 다시 다음 컴포넌트가 사용할 저장소로 보내는 방식이다.

Component들간의 interface는 저장소로만 구성되어 각 component들은 독립적이다. 이는 시스템을 파악하기 쉽게 해주고, 각 팀별로 자신의 component에만 집중할 수 있도록 해준다. 또한, component에 문제가 생겼을 때 다음 component는 문제가 생긴 component의 마지막 출력 결과를 사용해 계속해서 정상적으로 동작할 수 있으므로 architecture가 더욱 robust하게 된다.

하지만 적절한 모니터링의 부재로 인해 문제가 생긴 component를 감지하지 못한다면, 전체 시스템은 계속해서 과거의 데이터로만 동작하므로 성능이 하락하게 될 것이다.

## Select a Performance Measure
앞서 문제를 정의하였으므로 이제 모델의 성능 지표를 선택할 차례이다.

regression 문제에서 일반적으로 사용되는 성능 지표는 Root Meam Square Error(RMSE)이다. RMSE는 large error에 higher weight를 주는 방식이다.

RMSE는 다음의 식으로 계산된다.

- $\RMSE(\mathbf{X}, h) = \sqrt{\frac{1}{m} \sum\limits^m_{i=1} \big(h(\mathbf{x}^{(i)})-y^{(i)} \big)^2 }$

RMSE가 일반적으로 사용되지만, 몇가지 경우에는 다른 함수가 선호된다. 예를 들면, 많은 이상치(outlier)가 있는 경우, RMSE 대신에 Mean Absolute Error(MAE)를 사용할 수 있다.

MAE는 다음의 식으로 계산된다.

- $\MAE(\mathbf{X}, h) = \frac{1}{m} \sum\limits^m_{i=1} \Big| h(\mathbf{x}^{(i)})-y^{(i)} \Big|$

RMSE와 MAE는 target value vector와 prediction vector 사이의 거리를 측정하는 방법이다. 따라서, norm과 같은 다양한 거리 측정방법들도 사용할 수 있다.
- RMSE는 Euclidian norm($l_2$ norm)과 같다.
- MAE는 Manhattan norm($l_1$ norm)과 같다.
- 일반화된 식으로 나타내면, n개의 element를 가지는 vector $\mathbf{v}$의 $l_k$ norm은 $\|{\mathbf{v}}\|_k = \big( |v_0|^k + |v_1|^k + \cdots + |v_n|^k \big) ^ \frac{1}{k}$로 정의한다. $l_0$는 벡터 내의 0이 아닌 element들의 수를 의미하며, $l_\infty$는 벡터 내의 최대 절대 값이 된다.
- norm index가 커질 수록 큰 값에 집중하고 작은 값은 무시된다. 이는 RMSE가 MAE에 비해 outlier에 더 민감한 이유이다.

## Check the Assumptions
마지막으로, 지금까지의 가정들을 검사하는 것이 좋다.

예를 들어, 모델의 예측 결과가 전달될 뒤따르는 머신러닝 시스템이 price를 단순히 cheap, medium, expensive로만 구분한다면, 정확한 price가 중요하지 않으므로, regression 문제가 아니라 classification task가 되어야 한다.